In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
# zz


import pandas as pd


In [ ]:
Load = extractData(start=datetime(2019, 3, 1), end=datetime(2019, 3, 31))
print(len(Load))
Load.head()


In [ ]:
K = filterAirports(Load, ICAOTOP10)
K = calculateDelays(K)
# print(len(K))
K.head()


In [ ]:
P = K.drop(
    [
        "ACType",
        "ACOperator",
        "FlightType",
        "ActualDistanceFlown",
        "ECTRLID",
        "ADEPLat",
        "ADEPLong",
        "ADESLat",
        "ADESLong",
        "ActualOBT",
        "ActualAT",
        "ArrivalDelay",
        "DepartureDelay",
    ],
    axis=1,
)
P.sort_values("FiledOBT")
print(P.head(10))
print(P.shape)


In [ ]:
dep = P.groupby([pd.Grouper(key="FiledOBT", freq=f"{15}min"),]).agg(
    {"ADES": "count"}
)  # ["ADES"].value_counts().unstack(fill_value=0)
dep.head(30)
# dep.to_numpy()


In [ ]:
arr = P.groupby([ pd.Grouper(key="FiledAT", freq=f"{60}min"),
                "ADES"])["ADEP"].value_counts().unstack(fill_value=0)

mux = pd.MultiIndex.from_product([arr.index.get_level_values("FiledAT").unique(), arr.index.get_level_values("ADES").unique()])
# df = arr.pd.MultiIndex.sortlevel(level = ["FiledAT", "ADES"] , ascending= ICAOTOP10     )                      
df = arr.reindex(mux)
# df.iloc[:]["ADES"]

a  = (df.sort_index(level=0, ascending=True)
        .reindex(sorted(df.columns), axis=1)
        .fillna(0))
a = a.to_numpy()
maximum = np.zeros((10,10))

b = [a[i:i+10] for i in range(0,len(a), 10)]
b = np.array(b)
for i in range(len(a[0])):
        for j in range(len(a[0])):
                maximum[i][j] = max(b[:,i,j])
adj_matrices_demand = np.array([x / maximum for x in b])
